In [ ]:
%%bash
DATA_DIR=/tmp/asd-diagnosis

if [ ! -d $DATA_DIR ]; then
  mkdir -p $DATA_DIR
fi

wget https://minio.131.154.99.37.myip.cloud.infn.it/hackathon-data/asd-diagnosis/Harmonized_structural_features.csv -O $DATA_DIR/Harmonized_structural_features.csv &> .log
wget https://minio.131.154.99.37.myip.cloud.infn.it/hackathon-data/asd-diagnosis/Harmonized_functional_features.csv -O $DATA_DIR/Harmonized_functional_features.csv &> .log
wget https://minio.131.154.99.37.myip.cloud.infn.it/hackathon-data/asd-diagnosis/dict.csv -O $DATA_DIR/dict.csv &> .log

ls -lrth $DATA_DIR/*

# Autism Spectrum Disorders (ASD) diagnosis combining structural and functional Magnetic Resonance Imaging and Radiomics¶

For an introduction to Autism Spectrum Disorders (ASD) and to the dataset of this ML example please refer to the [first notebook](sMRI_fMRI_sep.ipynb). In that notebook structural and functional MRI features datasets were used separately. While here we are going to combine those datasets and evaluate if the ML model predictions have improved. 

## Usefull libraries to import

Here, you will have to read the data and pre-process them. First you need to import the necessary libraries and the apply a scaler to the data.

In [ ]:
import logging, os
logging.disable(logging.WARNING)
# https://stackoverflow.com/questions/40426502/is-there-a-way-to-suppress-the-messages-tensorflow-prints/40426709
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

import warnings
# https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

In [ ]:
EPOCH = 2

## Callback

A callback is an action passed to the model fit function which is performend while training the neural network. These actions allow you to modify certain parameters when a specific condition is met while training. These actions may be implemented before or after an epoch or batch is processed, when training begins or ends and when a specific condition is met. Callbacks can help with tasks like saving the model, reducing learning rates, stopping training early, or logging performance metrics. Here we implement two actions:

* EarlyStopping
* ReduceLROnPlateau

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

early_stop = EarlyStopping(monitor='loss', patience=10)

reduce_on_plateau = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=3,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0)

# Import dataframe and normalize with RobustScaler: DF_normalized

In [ ]:
import pandas as pd

path_to_data = '/tmp/asd-diagnosis/'

DF_struct = pd.read_csv(os.path.join(path_to_data,'Harmonized_structural_features.csv'))
DF_funct  = pd.read_csv(os.path.join(path_to_data,'Harmonized_functional_features.csv'))

In [ ]:
DF_struct = DF_struct.set_index('FILE_ID')
DF_funct = DF_funct.set_index('FILE_ID')
DF_funct= DF_funct.drop(['SITE', 'Database_Abide', 'AGE_AT_SCAN', 'DX_GROUP'], axis =1)
DF_merge = DF_struct.join(DF_funct,how='inner')
DF_merge = DF_merge.reset_index()
DF_merge.loc[DF_merge.DX_GROUP == -1, 'DX_GROUP'] = 0
DF_merge

In [ ]:
DF_normalized = DF_merge.drop(['SITE', 'Database_Abide', 'AGE_AT_SCAN', 'FILE_ID'], axis =1) #questa riga lasciamola che vanno tolte le feature ridondanti

In [ ]:
from sklearn.preprocessing import RobustScaler

#normalize data
RS_instance = RobustScaler()
DF_normalized.iloc[:,1:] = RS_instance.fit_transform(DF_normalized.iloc[:,1:].to_numpy())
DF_normalized

# ASD joint structural and functional model

In [ ]:
import tensorflow as tf
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

import logging
tf.get_logger().setLevel(logging.ERROR)

from tensorflow.keras.models import Model #Sequential, serve ?
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Concatenate, Dropout
from tensorflow.keras.regularizers import l1

In [ ]:
def structural_model():
    """ This function returns a model ...
    """
    input_data= Input(221,)
    x = Dense(32,  'relu', kernel_initializer='normal', kernel_regularizer=l1(0.01))(input_data)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(16, 'relu',kernel_initializer='normal', kernel_regularizer=l1(0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(8, 'relu',kernel_initializer='normal', kernel_regularizer=l1(0.01))(x)
    x = BatchNormalization()(x)
    #x = Dropout(0.1)(x)
    return Model(inputs=input_data, outputs=x)


def functional_model():
    """ This function returns a model ...
    """
    input_data= Input(5253,)
    x = Dense(64, 'relu', kernel_initializer= 'normal', kernel_regularizer=l1(0.01))(input_data)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(32, 'relu',kernel_initializer='normal', kernel_regularizer=l1(0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(16, 'relu',kernel_initializer='normal', kernel_regularizer=l1(0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(8, 'relu',kernel_initializer='normal', kernel_regularizer=l1(0.01))(x)
    x = BatchNormalization()(x)
    #x = Dropout(0.1)(x)
    return Model(inputs=input_data, outputs=x)


def joint_model(model_1, model_2):
    """ This function combines the output of two keras models
    model_1
    model_2

    returns: a new model which combines model_1 and model_2 by adding 3 dense layers
    """
    combined = Concatenate(axis=-1)([model_1.output, model_2.output])
    z = Dense(16, 'relu',  kernel_initializer= 'normal', kernel_regularizer=l1(0.01))(combined)
    z = BatchNormalization()(z)
    z = Dropout(0.2)(z)
    z = Dense(8, 'relu',  kernel_initializer= 'normal', kernel_regularizer=l1(0.01))(z)
    z = BatchNormalization()(z)
    z = Dropout(0.1)(z)
    z = Dense(1, 'sigmoid')(z) #, kernel_regularizer=l1(0.01))(z)
    return Model(inputs=[model_1.input, model_2.input], outputs = z)

**Descrivere il training**

In [ ]:
DF_normalized.head()  #il dataframe deve contenere sia le features strutturali che funzionali 

In [ ]:
import numpy as np

AUC = []
shap_values_per_cv_s =[]
shap_values_per_cv_f =[]
var_f = []
var_s = []
np.random.seed(1) # Reproducibility
n_cv = 2 #NUMERO DI KFOLD
rs_=13 
tprs = []
aucs = []
interp_fpr = np.linspace(0, 1, 100)

In [ ]:
import time

from tensorflow.keras.optimizers import SGD #stochastic gradient descent
from tensorflow.keras.backend import clear_session

# import sklearn 
from sklearn.model_selection import StratifiedKFold #train_test_split, KFold
from sklearn.metrics import roc_curve, auc

import shap # colpa sua "l'errore" qui sotto: /envs/ai4ni/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
import matplotlib.pyplot as plt

In [ ]:
cv = StratifiedKFold(n_splits=n_cv, shuffle=True, random_state=rs_) # Set random state

for train_index, test_index in cv.split(DF_normalized.iloc[:, 1:], DF_normalized.iloc[:, 0]):
    start = time.time()
    df_train, df_val = DF_normalized.iloc[train_index, :], DF_normalized.iloc[test_index, :]

    X_train_struct, X_test_struct = df_train.iloc[:, 1:222], df_val.iloc[:, 1:222]
    X_train_funct, X_test_funct = df_train.iloc[:, 222:], df_val.iloc[:, 222:]
    y_train, y_test = df_train.iloc[:, 0], df_val.iloc[:, 0]

        #load the model
    clear_session()
    mod_1 = structural_model
    mod_2 = functional_model
    model_joint = joint_model(mod_1(), mod_2())

        # Compile the model
    model_joint.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

        # Fit data to model
    history = model_joint.fit((X_train_struct, X_train_funct), y_train,
                          batch_size=64,
                          epochs=EPOCH,
                          verbose=0,
                          validation_data = ((X_test_struct, X_test_funct), y_test),
                          callbacks=[reduce_on_plateau]
                          )
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(1, len(acc)+1)
    
    #Train and validation accuracy
    plt.figure(figsize=(5, 5))
    plt.subplot(2, 2, 1)
    plt.plot(epochs_range, acc, label='Training ')
    plt.plot(epochs_range, val_acc, label='Validation ')
    plt.legend(loc='lower right')
    plt.title('Accuracy')
    #Train and validation loss
    plt.subplot(2, 2, 2)
    plt.plot(epochs_range, loss, label='Training ')
    plt.plot(epochs_range, val_loss, label='Validation ')
    plt.legend(loc='upper right')
    plt.title(' Loss')
    plt.show()
    
    # prepare for SHAP
    X_train_struct_SHAP = X_train_struct.to_numpy()
    X_test_struct_SHAP = X_test_struct.to_numpy()
    X_train_funct_SHAP = X_train_funct.to_numpy()
    X_test_funct_SHAP = X_test_funct.to_numpy()

    # Use SHAP to explain predictions
    explainer = shap.GradientExplainer(model_joint, [X_train_struct_SHAP, X_train_funct_SHAP])
    shap_values = explainer.shap_values([X_test_struct_SHAP, X_test_funct_SHAP])
    # nsamples maggario permettono un calcolo più accurato - rappresenta il numero di perturbazioni 200 def nsamples = 200
    #print(shap_values)
    #  SHAP information per fold
    shap_values_per_cv_s.append(shap_values[0]) #  221 features, appendo un array di dimensione 138/139,  221, 1
    shap_values_per_cv_f.append(shap_values[1]) # 5253 features, appendo un array di dimensione 138/139, 5253, 1

    print("len(shap_values[0]) ",len(shap_values[0]))
    print("shap_values[0].shape ",shap_values[0].shape)
    print("len(shap_values[1]) ",len(shap_values[1]))
    print("shap_values[1].shape ",shap_values[1].shape)

    #print(X_test_struct.shape)
        ###########################################################
    _, val_acc = model_joint.evaluate((X_test_struct, X_test_funct), y_test, verbose=0)
    acc.append(val_acc)

        #Compute Receiver operating characteristic (ROC)
    i=0
    preds = model_joint.predict((X_test_struct, X_test_funct), verbose=1)
    fpr, tpr, _ = roc_curve(y_test, preds)
    roc_auc = auc(fpr, tpr)
    interp_tpr = np.interp(interp_fpr, fpr, tpr)
    tprs.append(interp_tpr)
    AUC.append(roc_auc)
    i += 1
    print('---------------------AUC------------------', roc_auc)
    end = time.time()
    print('----------------------T-------------------', end - start)
    print("")

In [ ]:
plt.figure()
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
      label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(interp_fpr, mean_tpr)
std_auc = np.std(AUC)
plt.plot(interp_fpr, mean_tpr, color='b',
        label=f'Mean ROC (AUC = {mean_auc:.2f} $\pm$ {std_auc:.2f})',
        lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(interp_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False Positive Rate',fontsize=18)
plt.ylabel('True Positive Rate',fontsize=18)
plt.title('Joint Fusion model',fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()

In [ ]:
# == Provide average scores ==
print(len(AUC))
print(f'AUC:{np.mean(AUC)} (+- {np.std(AUC)})')
print(f'accuracy: {np.mean(acc)} (+- {np.std(acc)})')

In [ ]:
# Establish lists to keep average Shap values
average_shap_values_s = []
average_shap_values_f = []

for i in range(0, len(AUC)):
    df_per_obs = shap_values_per_cv_s[i].copy()
    df_per_obs = np.absolute(df_per_obs)
    average_shap_values_s.append(df_per_obs.mean(axis=0))

    df_per_f = shap_values_per_cv_f[i].copy()
    df_per_f = np.absolute(df_per_f)
    average_shap_values_f.append(df_per_f.mean(axis=0))

fold_s = np.transpose(np.array(average_shap_values_s)[...,0])
fold_f = np.transpose(np.array(average_shap_values_f)[...,0])

# SHAP values Joint model: NORMALIZATION SHAP VALUES 

In [ ]:
fold_s = pd.DataFrame.from_dict(fold_s)
fold_f = pd.DataFrame.from_dict(fold_f)

In [ ]:
fold_s = fold_s.iloc[:,1:]
fold_f = fold_f.iloc[:,1:]

In [ ]:
s = 221/(2*(221 + 5253))
f = 5253/(2*(221 + 5253))

In [ ]:
# NORMNALIZZAZIONE STRUTTURALE

fold_s_n = (fold_s/fold_s.sum(axis=0))*s*100
plot = fold_s_n.mean(axis=1).values
strutt = pd.DataFrame(plot, index = DF_normalized.iloc[:, 1:222].columns )
strutt['std']=fold_s_n.std(axis=1).values
strutt = strutt.sort_values(by=0, ascending=False)
strutt

In [ ]:
# NORMNALIZZAZIONE FUNZIONALE

norm = fold_f.sum(axis=0)
fold_f_n = (fold_f/fold_f.sum(axis=0))*f*100
plot_f = fold_f_n.mean(axis=1).values
func = pd.DataFrame(plot_f, index = DF_normalized.iloc[:, 222:].columns )
func['std']=fold_f_n.std(axis=1).values
func = func.sort_values(by=0, ascending=False)
'''func = func.iloc[:35, :]
func.to_excel(path+'risultati_funzionali.xlsx')'''
func

In [ ]:
# Non capisco perche' si vada a leggere un xlsx

#func =pd.read_excel(path+'risultati_funzionali.xlsx')
#func['name'] = func['r1'].astype(str) + ' - ' + func['r2'].astype(str)

#del func['r1']
#del func['r2']
#del func['Unnamed: 0']
#func = func.set_index('name')

In [ ]:
all_m_s =  pd.concat([strutt,func])
all_m_s = all_m_s.sort_values(by=0, ascending=False)
all_m_s

In [ ]:
th99 = all_m_s.iloc[:, 0].quantile(0.995)
th99

## 99-95 esimo percentile

In [ ]:
all_m_s[all_m_s[0] >= th99][0]

In [ ]:
ax2 = plt.figure(figsize=(40, 50))
ax2 = all_m_s[all_m_s[0] >= th99][0].plot(kind="barh", figsize=(10,10))
ax2.invert_yaxis()
plt.xlabel("mean(|SHAP value|)")

## Cohen d coeff

The Cohen's *d* coefficient is a statistical measure used to quantify the *effect size* between two groups, indicating the standardized difference between their means. It is commonly used in psychology, social sciences, and other fields to assess the difference between two sample groups.

### Formula

The formula for Cohen’s *d* is:
$$
d = \frac{\bar{X}_1 - \bar{X}_2}{s}
$$
where: $\bar{X}_1$ and $\bar{X}_2$ are the means of the two groups. $s$ is the pooled standard deviation of the two groups, calculated as:
$$
s = \sqrt{\frac{(n_1 - 1)s_1^2 + (n_2 - 1)s_2^2}{n_1 + n_2 - 2}}
$$
where: $n_1$ and $n_2$ are the sample sizes of the two groups. $s_1$ and $s_2$ are the standard deviations of the two groups.

### Interpretation
Cohen's *d* provides a way to interpret the magnitude of the difference, regardless of the scale of the data, making it easier to compare across studies. Common interpretations are:
- **0.2** - Small effect size
- **0.5** - Medium effect size
- **0.8** or higher - Large effect size

These are general guidelines, and the interpretation can vary by field. Cohen's *d* is particularly helpful because it puts the difference in a standardized context, allowing researchers to understand the size of an effect without being influenced by sample size alone.

In [ ]:
def Cohen_d(g1, g2, f):
    """Function to compute the Cohen's d Coefficient.

    g1: infered results by predictor 1
    g2: infered results by predictor 2
    f:  predicted class ?

    it returns the 'd' value of agreement
    """
    n1 = len(g1)                   # number of data in g1
    n2 = len(g2)                   # number of data in g2
    N = n1 + n2                    # total number of data
    Scores1 = g1[f].dropna()       # remove nan results
    Scores2 = g2[f].dropna()
    var1 = Scores1.var()           # compute the variance over the dataset
    var2 = Scores2.var()
    mean1 = Scores1.mean()         # compute the mean result
    mean2 = Scores2.mean()
    sp = (((n1 - 1)*var1 + (n2 - 1)*var2) / (N - 2))**0.5  # A me sembra tanto un confronto tra gaussiane
    d = (mean1 - mean2) / sp
    return d

In [ ]:
controls = DF_normalized[DF_normalized.DX_GROUP==0]
ASD =  DF_normalized[DF_normalized.DX_GROUP==1]
list_f = all_m_s.iloc[:].index.tolist()
score_df = []
for item in list_f:
    score =  Cohen_d(ASD, controls, item)
    score_df.append(score)
    #print(item, score)
all_m_s['cohen']=score_df

In [ ]:
all_m_s

## Find region's coord in HO

In [ ]:
from nilearn import datasets, plotting

path_to_data = '/tmp/asd-diagnosis/'
dict_f = pd.read_csv(path_to_data + 'dict.csv')
df2 = pd.read_csv(path_to_data + 'functional_features.csv')
df2 = df2.set_index('F')

perc_95 = dict_f['Unnamed: 0'][0:25]
perc_95
lista_con = df2.loc[perc_95]
lista_con
all_m_s.iloc[:28, :]
'''#features ASD>TD
lista_conn = lista_con.drop(731)
lista_conn = lista_conn.drop(3254)
lista_conn = lista_conn.drop(3295)

lista_conn = lista_conn.drop(3004)
lista_conn = lista_conn.drop(1737)
lista_conn = lista_conn.drop(4545)
lista_conn = lista_conn.drop(4935)
lista_conn = lista_conn.drop(1455)
lista_conn = lista_conn.drop(395)'''


#features ASD>TD
lista_conn = lista_con.loc[[731, 3254, 3295, 3004, 1737, 4545, 4935, 1455, 395]]
lista_conn
l1 = lista_conn['r1'].to_list()
l2 = lista_conn['r2'].to_list()
l = l1 + l2
sam_list = list(set(l))
sam_list
len(sam_list)
region_c = []
label_c = []
#load HO atlas
atlas_ho = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm')  #sub-maxprob-thr50-2mm - cortl-maxprob-thr25-2mm
atlas_file = atlas_ho.maps

# Load labels for each atlas region
labels = atlas_ho.labels[1:]
coordinates = plotting.find_parcellation_cut_coords(labels_img=atlas_file)
print(len(labels))

atlas_ho = datasets.fetch_atlas_harvard_oxford('cortl-maxprob-thr25-2mm')
atlas_file = atlas_ho.maps
# Load labels for each atlas region
labels = atlas_ho.labels[1:]
coordinates = plotting.find_parcellation_cut_coords(labels_img=atlas_file)
print(len(labels))

for i, e in enumerate(labels):
  for j, n in enumerate(sam_list):
    if e == n:
      region_c.append(coordinates[i])
      label_c.append(e)

len(label_c)

## PLOT connectoma

In [ ]:
from matplotlib import colormaps as cm

#print("lista_conn")
#print(lista_conn)
#print("")

for i in range(0, len(label_c)):
    print("i, region_c[i],label_c[i]")
    print(i, region_c[i],label_c[i])

index_1 = []
for j, n in enumerate(l1):
    for i, e in enumerate(label_c):
        if n == e:
            index_1.append(i)

index_2=[]
for j, n in enumerate(l2):
    for i, e in enumerate(label_c):
        if n == e:
            index_2.append(i)

print("")
print("lista_conn['i1']",lista_conn['r1'])
print("lista_conn['i2']",lista_conn['r2'])

#lista_conn['r1'] = index_1
#lista_conn['r2'] = index_2
#lista_conn

mat =np.zeros((len(region_c), len(region_c)))

for index, (value1, value2) in enumerate(zip(index_1, index_2)):
    #print(index, value1 , value2)
    mat[value1][value2] = 1
mat

mat = mat + mat.T
coordinates = np.array(region_c)  # 3D coordinates of parcels

color_dict = {}
cmap = cm.get_cmap('tab20')

for rsn, c in zip(label_c, cmap.colors):
    color_dict[rsn] = tuple(c)



node_color = []
for nw in label_c:
    node_color.append(color_dict[nw])

coords = coordinates

N = len(coords)

plotting.plot_connectome(mat,
                         coords,
                         title='ASD>TD',
                         node_color=node_color,
                          display_mode="lyrz",
                         edge_kwargs = {"linewidth":1.7, "color": 'red'})

from matplotlib.lines import Line2D

legend_elements = []
for k,v in color_dict.items():
    legend_elements.append(Line2D([0], [0], marker='o', color=v, label=k,
                          markerfacecolor=v, markersize=5))
# Create the figure
fig, ax = plt.subplots()
ax.axis("off")
ax.legend(handles=legend_elements, loc='center')